## 필요 라이브러리 설치

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.8/898.8 kB 46.6 MB/s eta 0:00:00


In [ ]:
!pip install -U albumentations

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.0/66.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 632.7/632.7 kB 41.1 MB/s eta 0:00:00
  Attempting uninstall: albucore
    Found existing installation: albucore 0.0.19
    Uninstalling albucore-0.0.19:
      Successfully uninstalled albucore-0.0.19
  Attempting uninstall: albumentations
    Found existing installation: albumentations 1.4.20
    Uninstalling albumentations-1.4.20:
      Successfully uninstalled albumentations-1.4.20


In [ ]:
!pip install albumentations[imgaug]
!pip install -U albumentations pytorch

  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pytorch
  Running setup.py clean for pytorch
Failed to build pytorch
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (pytorch)


In [ ]:
!pip install torch torchvision torchaudio


## 학습 데이터 로드

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

In [ ]:
# 데이터 경로 설정
base_dir = '/content/drive/MyDrive/Seg_dataset'

train_images_dir = f"{base_dir}/images/train"
val_images_dir = f"{base_dir}/images/val"
train_labels_dir = f"{base_dir}/labels/train"
val_labels_dir = f"{base_dir}/labels/val"

# 데이터 경로 출력
print(f"Train Images: {train_images_dir}")
print(f"Validation Images: {val_images_dir}")
print(f"Train Labels: {train_labels_dir}")
print(f"Validation Labels: {val_labels_dir}")


Train Images: /content/drive/MyDrive/Seg_dataset/images/train
Validation Images: /content/drive/MyDrive/Seg_dataset/images/val
Train Labels: /content/drive/MyDrive/Seg_dataset/labels/train
Validation Labels: /content/drive/MyDrive/Seg_dataset/labels/val


In [ ]:
# 파일 개수를 확인하는 함수
def count_files_in_directory(directory, extensions=None):
    if os.path.exists(directory):
        return len([file for file in os.listdir(directory) if not extensions or file.endswith(extensions)])
    else:
        return 0

# 이미지와 레이블 파일 개수 확인
train_images_count = count_files_in_directory(train_images_dir, ('.jpg', '.jpeg', '.png'))
val_images_count = count_files_in_directory(val_images_dir, ('.jpg', '.jpeg', '.png'))
train_labels_count = count_files_in_directory(train_labels_dir, ('.txt',))
val_labels_count = count_files_in_directory(val_labels_dir, ('.txt',))

print(f"Train Images: {train_images_count}, Train Labels: {train_labels_count}")
print(f"Validation Images: {val_images_count}, Validation Labels: {val_labels_count}")

Train Images: 894, Train Labels: 894
Validation Images: 256, Validation Labels: 256


In [ ]:
# YAML 파일 생성
data_yaml = f"""
train: {train_images_dir}
val: {val_images_dir}

nc: 2  # 클래스 개수
names: ['person', 'bed']  # 클래스 이름
"""

# YAML 파일 저장
yaml_path = '/content/custom_data.yaml'
with open(yaml_path, 'w') as f:
    f.write(data_yaml)

print(f"YAML 파일 생성 완료: {yaml_path}")


YAML 파일 생성 완료: /content/custom_data.yaml


## 모델 학습 및 저장

In [ ]:
from ultralytics import YOLO

from albumentations import (
    Compose, Resize, RandomBrightnessContrast, HorizontalFlip,
    Rotate, GaussianBlur, CLAHE, HueSaturationValue
)
from torchvision.transforms import ToTensor  # PyTorch 전용


def get_transforms():
    return Compose([
        Resize(640, 640),  # 이미지 크기 조정
        RandomBrightnessContrast(p=0.3),  # 밝기/대비 조정
        HorizontalFlip(p=0.5),  # 수평 플립
        Rotate(limit=45, p=0.5),  # ±45도 회전
        GaussianNoise(p=0.3),  # 잡음 추가
        HueSaturationValue(p=0.3),  # 색조/채도 조정
        CLAHE(p=0.3),  # 대비 제한 히스토그램 평활화
        ToTensorV2()  # PyTorch 텐서로 변환
    ])

# YOLOv8 Segmentation 모델 로드
model = YOLO('yolov8n-seg.pt')  # Segmentation 모델
pose_model = YOLO('yolov8n-pose.pt')

# 모델 학습
model.train(
    data=yaml_path,           # 데이터셋 경로(YAML 파일)
    epochs=100,                # 학습 반복 횟수
    imgsz=640,                # 입력 이미지 크기
    batch=16,                  # 배치 크기
    name='custom_seg_model',  # 모델 저장 경로
    device=0                  # GPU 사용 (0번 GPU)
)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.74M/6.74M [00:00<00:00, 337MB/s]


100%|██████████| 6.52M/6.52M [00:00<00:00, 395MB/s]

Ultralytics 8.3.48 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=/content/custom_data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=custom_seg_model, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=Tr

100%|██████████| 755k/755k [00:00<00:00, 102MB/s]


...

                   all        256        926      0.691      0.492      0.543      0.337      0.679      0.492      0.512      0.277



100 epochs completed in 0.337 hours.
Optimizer stripped from runs/segment/custom_seg_model/weights/last.pt, 6.8MB
Optimizer stripped from runs/segment/custom_seg_model/weights/best.pt, 6.8MB

Validating runs/segment/custom_seg_model/weights/best.pt...
Ultralytics 8.3.48 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
YOLOv8n-seg summary (fused): 195 layers, 3,258,454 parameters, 0 gradients, 12.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:02<00:00,  2.75it/s]


                   all        256        926      0.653      0.515      0.542      0.337      0.636      0.503      0.512      0.278
                person        256        495      0.712      0.576      0.606       0.39      0.689      0.558      0.578      0.326
                   bed        256        431      0.594      0.455      0.478      0.285      0.584      0.448      0.446       0.23
Speed: 0.2ms preprocess, 1.1ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to runs/segment/custom_seg_model


ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7b364f427310>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.04104

In [ ]:
import shutil

# 모델 저장 경로
trained_model_path = '/content/runs/segment/custom_seg_model/weights/best.pt'

# Google Drive 저장 경로
drive_save_path = '/content/drive/MyDrive/Seg_dataset'

# 모델 파일 복사
shutil.copy(trained_model_path, drive_save_path)

print(f"Model saved to: {drive_save_path}")


Model saved to: /content/drive/MyDrive/Seg_dataset


## 테스트 및 후처리

### keypoint와 mask 출력 및 keypoint 뼈대 연결

In [ ]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO

# YOLOv8 Segmentation 모델 로드
model = YOLO('/content/drive/MyDrive/Seg_dataset/best.pt')  # Segmentation 모델
pose_model = YOLO('yolov8n-pose.pt')



# 중심점 계산 함수
def calculate_centroid(mask):
    indices = np.argwhere(mask)  # 마스크에서 1인 픽셀 좌표를 가져옴
    if indices.size == 0:
        return None  # 비어 있는 경우 None 반환
    return np.mean(indices, axis=0)  # 좌표 평균 계산

# 위험 감지 함수
def detect_risk(pose_keypoints, bed_mask, image_shape, original_shape):
    if pose_keypoints is None or len(pose_keypoints) == 0 or len(pose_keypoints[0]) < 7:
        return "Cannot assess risk. Missing or incomplete keypoints."

    if bed_mask is None:
        return "Cannot assess risk. Missing bed mask."

    # 침대 가장자리 계산
    bed_mask_resized = cv2.resize(bed_mask, (image_shape[1], image_shape[0]))
    bed_edges = np.argwhere(bed_mask_resized > 0)
    if bed_edges.size == 0:
        return "Cannot assess risk. Bed mask is empty."

    bed_min_y, bed_max_y = np.min(bed_edges[:, 0]), np.max(bed_edges[:, 0])
    bed_min_x, bed_max_x = np.min(bed_edges[:, 1]), np.max(bed_edges[:, 1])

    # 얼굴 Keypoints 확인 (코, 두 눈, 두 귀)
    face_keypoints = pose_keypoints[0][:5]  # 얼굴 Keypoints
    for idx, keypoint in enumerate(face_keypoints):
        x, y = keypoint[:2]
        x_original = x * (image_shape[1] / original_shape[1])
        y_original = y * (image_shape[0] / original_shape[0])

        # 침대 내부 여부 확인
        if not (bed_min_x <= x_original <= bed_max_x and bed_min_y <= y_original <= bed_max_y):
            return f"Warning: Baby's face keypoint {idx} is outside the bed!"

    # 코, 가슴, 배, 그리고 양 눈 Keypoints 추출
    nose, left_eye, right_eye = pose_keypoints[0][0], pose_keypoints[0][1], pose_keypoints[0][2]
    chest, stomach = pose_keypoints[0][5], pose_keypoints[0][6]

    # 얼굴이 정면을 보고 있는지 확인
    eye_center_x = (left_eye[0] + right_eye[0]) / 2
    eye_center_y = (left_eye[1] + right_eye[1]) / 2
    if abs(nose[0] - eye_center_x) < image_shape[1] * 0.05 and abs(nose[1] - eye_center_y) < image_shape[0] * 0.05:
        return "No risk detected. Baby's face is facing forward."

    # 엎드린 자세 확인
    if chest[1] > nose[1] and stomach[1] > nose[1]:
        return "Warning: Baby is lying face down!"

    # 코와 가슴 Keypoints 간 거리 계산
    vertical_distance = abs(nose[1] - chest[1])
    vertical_threshold = image_shape[0] * 0.1  # 이미지 높이의 10% 임계값
    if vertical_distance < vertical_threshold:
        return "Warning: Baby's head is dangerously close to their chest!"

    return "No risk detected."


# COCO Keypoint 연결 정의
pose_connections = [
    (0, 1), (0, 2), (1, 3), (2, 4),  # 머리 연결
    (5, 6), (5, 7), (7, 9), (6, 8), (8, 10),  # 팔 연결
    (11, 12), (11, 13), (13, 15), (12, 14), (14, 16)  # 다리 연결
]

# 뼈대 시각화 함수
def draw_pose_skeleton(image, keypoints, connections, img_shape, orig_shape):
    for connection in connections:
        start_idx, end_idx = connection
        if start_idx >= len(keypoints) or end_idx >= len(keypoints):
            continue

        # 시작점과 끝점 좌표 계산
        start_x, start_y, start_conf = keypoints[start_idx]
        end_x, end_y, end_conf = keypoints[end_idx]
        if start_conf < 0.5 or end_conf < 0.5:  # 신뢰도가 낮으면 생략
            continue

        start_x = int(start_x * (img_shape[1] / orig_shape[1]))
        start_y = int(start_y * (img_shape[0] / orig_shape[0]))
        end_x = int(end_x * (img_shape[1] / orig_shape[1]))
        end_y = int(end_y * (img_shape[0] / orig_shape[0]))

        # 유효한 키포인트만 연결
        if start_x > 0 and start_y > 0 and end_x > 0 and end_y > 0:
            cv2.line(image, (start_x, start_y), (end_x, end_y), (0, 255, 255), 2)  # 노란색 선

    return image

# 테스트 이미지 경로
test_images_dir = '/content/drive/MyDrive/Seg_dataset/Kakao'
image_paths = [os.path.join(test_images_dir, img) for img in os.listdir(test_images_dir) if img.endswith(('.jpg', '.png'))]


# 테스트 실행
for image_path in image_paths:
    # 이미지 로드
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Unable to load image {image_path}")
        continue

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 모델 추론
    try:
        seg_results = model(image, conf=0.5, iou=0.4)  # IOU 임계값 설정
        pose_results = pose_model(image)  # Pose 추론
    except Exception as e:
        print(f"Error during inference: {e}")
        continue

    # Segmentation Masks 및 Keypoints 추출
    masks = seg_results[0].masks.data.cpu().numpy() if seg_results[0].masks is not None else []
    pose_keypoints = pose_results[0].keypoints.data.cpu().numpy() if pose_results[0].keypoints is not None else None

    # 침대 마스크 필터링
    bed_class_id = 1  # 침대 클래스 ID
    bed_mask = None
    if seg_results[0].boxes is not None:
        classes = seg_results[0].boxes.cls.cpu().numpy()
        bed_indices = np.where(classes == bed_class_id)[0]
        if len(bed_indices) > 0:
            bed_mask = masks[bed_indices[0]]

    # Segmentation 마스크 시각화
    mask_overlay = np.zeros_like(image, dtype=np.uint8)
    if bed_mask is not None:
        bed_mask_resized = cv2.resize(bed_mask, (image.shape[1], image.shape[0]))
        mask_bool = bed_mask_resized > 0.5
        mask_overlay[mask_bool] = [255, 0, 0]  # 빨간색으로 표시

        # 침대 가장자리 시각화
        bed_edges = np.argwhere(mask_bool)
        if bed_edges.size > 0:
            bed_min_y, bed_max_y = np.min(bed_edges[:, 0]), np.max(bed_edges[:, 0])
            bed_min_x, bed_max_x = np.min(bed_edges[:, 1]), np.max(bed_edges[:, 1])
            cv2.rectangle(mask_overlay, (bed_min_x, bed_min_y), (bed_max_x, bed_max_y), (255, 255, 0), 2)

    # Keypoints 및 뼈대 시각화
    blended_image = cv2.addWeighted(image, 0.7, mask_overlay, 0.3, 0)
    if pose_keypoints is not None and len(pose_keypoints[0]) > 0:
        keypoints = pose_keypoints[0]  # 첫 번째 사람의 키포인트 사용
        # 뼈대 시각화
        blended_image = draw_pose_skeleton(blended_image, keypoints, pose_connections, image.shape, pose_results[0].orig_shape)

        # 개별 키포인트 시각화
        for keypoint in keypoints:
            x, y, conf = keypoint
            if conf < 0.5:  # 신뢰도가 낮으면 생략
                continue
            x_original = int(x * (image.shape[1] / pose_results[0].orig_shape[1]))
            y_original = int(y * (image.shape[0] / pose_results[0].orig_shape[0]))
            cv2.circle(blended_image, (x_original, y_original), 5, (0, 255, 0), -1)  # 초록색 점

    # 위험 감지
    risk_message = detect_risk(pose_keypoints, bed_mask, image.shape, pose_results[0].orig_shape)

    # 결과 시각화
    plt.figure(figsize=(10, 10))
    plt.imshow(blended_image)
    plt.title(f"{os.path.basename(image_path)} - {risk_message}")
    plt.axis("off")
    plt.show()

    # 이미지 파일 이름 및 상태 출력
    print(f"Processed Image: {os.path.basename(image_path)}")
    print(risk_message)


Output hidden; open in https://colab.research.google.com to view.

### 위험단계 표시(1단계: 가장자리로 이동, 2단계: 주요 kp 침대 테투리에서 벗어나는 경우)

In [ ]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO

# YOLOv8 Segmentation 모델 로드
model = YOLO('/content/drive/MyDrive/Seg_dataset/best.pt')  # Segmentation 모델
pose_model = YOLO('yolov8n-pose.pt')

# 중심점 계산 함수
def calculate_centroid(mask):
    indices = np.argwhere(mask)  # 마스크에서 1인 픽셀 좌표를 가져옴
    if indices.size == 0:
        return None  # 비어 있는 경우 None 반환
    return np.mean(indices, axis=0)  # 좌표 평균 계산

# 위험 감지 함수 (위험 단계 1, 2 추가)
def detect_risk(pose_keypoints, bed_mask, image_shape, original_shape):
    if pose_keypoints is None or len(pose_keypoints) == 0 or len(pose_keypoints[0]) < 7:
        return "Cannot assess risk. Missing or incomplete keypoints."

    if bed_mask is None:
        return "Cannot assess risk. Missing bed mask."

    # 침대 가장자리 계산
    bed_mask_resized = cv2.resize(bed_mask, (image_shape[1], image_shape[0]))
    bed_edges = np.argwhere(bed_mask_resized > 0)
    if bed_edges.size == 0:
        return "Cannot assess risk. Bed mask is empty."

    bed_min_y, bed_max_y = np.min(bed_edges[:, 0]), np.max(bed_edges[:, 0])
    bed_min_x, bed_max_x = np.min(bed_edges[:, 1]), np.max(bed_edges[:, 1])

    # 위험 단계 1: 침대 가장자리 접근
    danger_level_1 = False
    edge_threshold = 10  # 침대 가장자리로 간주할 임계값 (10픽셀)

    # 위험 단계 2: 머리/손발이 침대 경계 밖으로 나감
    danger_level_2 = False
    keypoints_to_check = [0, 9, 10, 15, 16]  # 머리(0), 손목(9, 10), 발목(15, 16)

    for idx, keypoint in enumerate(pose_keypoints[0]):
        x, y, conf = keypoint
        if conf < 0.5:  # 신뢰도 낮은 Keypoints는 생략
            continue
        x_original = x * (image_shape[1] / original_shape[1])
        y_original = y * (image_shape[0] / original_shape[0])

        # 경계 밖 여부 확인
        if not (bed_min_x <= x_original <= bed_max_x and bed_min_y <= y_original <= bed_max_y):
            if idx in keypoints_to_check:  # 머리/손발이 경계 밖으로 나감
                danger_level_2 = True
                return f"Warning Level 2: Keypoint {idx} is outside the bed!"
        else:
            # 침대 가장자리로 접근 여부 확인
            if (
                (bed_min_x + edge_threshold <= x_original <= bed_max_x - edge_threshold)
                and (bed_min_y + edge_threshold <= y_original <= bed_max_y - edge_threshold)
            ):
                continue
            else:
                danger_level_1 = True

    # 위험 단계 결과 반환
    if danger_level_2:
        return "Warning Level 2: Baby is at high risk!"
    if danger_level_1:
        return "Warning Level 1: Baby is near the edge of the bed!"

    return "No risk detected. Baby is safe."

# COCO Keypoint 연결 정의
pose_connections = [
    (0, 1), (0, 2), (1, 3), (2, 4),  # 머리 연결
    (5, 6), (5, 7), (7, 9), (6, 8), (8, 10),  # 팔 연결
    (11, 12), (11, 13), (13, 15), (12, 14), (14, 16)  # 다리 연결
]

# 뼈대 시각화 함수
def draw_pose_skeleton(image, keypoints, connections, img_shape, orig_shape):
    for connection in connections:
        start_idx, end_idx = connection
        if start_idx >= len(keypoints) or end_idx >= len(keypoints):
            continue

        # 시작점과 끝점 좌표 계산
        start_x, start_y, start_conf = keypoints[start_idx]
        end_x, end_y, end_conf = keypoints[end_idx]
        if start_conf < 0.5 or end_conf < 0.5:  # 신뢰도가 낮으면 생략
            continue

        start_x = int(start_x * (img_shape[1] / orig_shape[1]))
        start_y = int(start_y * (img_shape[0] / orig_shape[0]))
        end_x = int(end_x * (img_shape[1] / orig_shape[1]))
        end_y = int(end_y * (img_shape[0] / orig_shape[0]))

        # 유효한 Keypoint만 연결
        if start_x > 0 and start_y > 0 and end_x > 0 and end_y > 0:
            cv2.line(image, (start_x, start_y), (end_x, end_y), (0, 255, 255), 2)  # 노란색 선

    return image

# 테스트 이미지 경로
test_images_dir = '/content/drive/MyDrive/Seg_dataset/Kakao'
image_paths = [os.path.join(test_images_dir, img) for img in os.listdir(test_images_dir) if img.endswith(('.jpg', '.png'))]

# 테스트 실행
for image_path in image_paths:
    # 이미지 로드
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Unable to load image {image_path}")
        continue

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 모델 추론
    try:
        seg_results = model(image, conf=0.5, iou=0.4)  # IOU 임계값 설정
        pose_results = pose_model(image)  # Pose 추론
    except Exception as e:
        print(f"Error during inference: {e}")
        continue

    # Segmentation Masks 및 Keypoints 추출
    masks = seg_results[0].masks.data.cpu().numpy() if seg_results[0].masks is not None else []
    pose_keypoints = pose_results[0].keypoints.data.cpu().numpy() if pose_results[0].keypoints is not None else None

    # 침대 마스크 필터링
    bed_class_id = 1  # 침대 클래스 ID
    bed_mask = None
    if seg_results[0].boxes is not None:
        classes = seg_results[0].boxes.cls.cpu().numpy()
        bed_indices = np.where(classes == bed_class_id)[0]
        if len(bed_indices) > 0:
            bed_mask = masks[bed_indices[0]]

    # Segmentation 마스크 시각화
    mask_overlay = np.zeros_like(image, dtype=np.uint8)
    if bed_mask is not None:
        bed_mask_resized = cv2.resize(bed_mask, (image.shape[1], image.shape[0]))
        mask_bool = bed_mask_resized > 0.5
        mask_overlay[mask_bool] = [255, 0, 0]  # 빨간색으로 표시

        # 침대 가장자리 시각화
        bed_edges = np.argwhere(mask_bool)
        if bed_edges.size > 0:
            bed_min_y, bed_max_y = np.min(bed_edges[:, 0]), np.max(bed_edges[:, 0])
            bed_min_x, bed_max_x = np.min(bed_edges[:, 1]), np.max(bed_edges[:, 1])
            cv2.rectangle(mask_overlay, (bed_min_x, bed_min_y), (bed_max_x, bed_max_y), (255, 255, 0), 2)

    # Keypoints 및 뼈대 시각화
    blended_image = cv2.addWeighted(image, 0.7, mask_overlay, 0.3, 0)
    if pose_keypoints is not None and len(pose_keypoints[0]) > 0:
        keypoints = pose_keypoints[0]  # 첫 번째 사람의 Keypoint 사용
        # 뼈대 시각화
        blended_image = draw_pose_skeleton(blended_image, keypoints, pose_connections, image.shape, pose_results[0].orig_shape)

        # 개별 Keypoint 시각화
        for keypoint in keypoints:
            x, y, conf = keypoint
            if conf < 0.5:  # 신뢰도가 낮으면 생략
                continue
            x_original = int(x * (image.shape[1] / pose_results[0].orig_shape[1]))
            y_original = int(y * (image.shape[0] / pose_results[0].orig_shape[0]))
            cv2.circle(blended_image, (x_original, y_original), 5, (0, 255, 0), -1)  # 초록색 점

    # 위험 감지
    risk_message = detect_risk(pose_keypoints, bed_mask, image.shape, pose_results[0].orig_shape)

    # 결과 시각화
    plt.figure(figsize=(10, 10))
    plt.imshow(blended_image)
    plt.title(f"{os.path.basename(image_path)} - {risk_message}")
    plt.axis("off")
    plt.show()

    # 이미지 파일 이름 및 상태 출력
    print(f"Processed Image: {os.path.basename(image_path)}")
    print(risk_message)


Output hidden; open in https://colab.research.google.com to view.

### 침대 객체를 인식 못하거나 사람보다 작은 크기로 인식하면, person을 제외한 나머지 부분을 침대로 인식

In [ ]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO

# YOLOv8 Segmentation 모델 로드
model = YOLO('/content/drive/MyDrive/Seg_dataset/best.pt')  # Segmentation 모델
pose_model = YOLO('yolov8n-pose.pt')

# 테스트 이미지 경로
test_images_dir = '/content/drive/MyDrive/baby_Prone'
image_paths = [os.path.join(test_images_dir, img) for img in os.listdir(test_images_dir) if img.endswith(('.jpg', '.png'))]


# 중심점 계산 함수
def calculate_centroid(mask):
    indices = np.argwhere(mask)  # 마스크에서 1인 픽셀 좌표를 가져옴
    if indices.size == 0:
        return None  # 비어 있는 경우 None 반환
    return np.mean(indices, axis=0)  # 좌표 평균 계산

# 침대 마스크 보정 함수
def refine_bed_mask(image_shape, bed_mask, person_mask):
    """
    침대 마스크가 없거나 작을 경우 person 마스크를 제외한 나머지를 침대 마스크로 인식.
    """
    if bed_mask is None or np.sum(bed_mask) < 0.1 * (image_shape[0] * image_shape[1]):  # 10% 이하인 경우
        print("Refining bed mask using person mask...")
        bed_mask = np.ones(image_shape[:2], dtype=np.uint8)  # 전체 영역을 1로 초기화
        if person_mask is not None:
            person_mask_resized = cv2.resize(person_mask, (image_shape[1], image_shape[0]))
            bed_mask[person_mask_resized > 0.5] = 0  # person 영역 제외
    return bed_mask

# 위험 감지 함수 (위험 단계 1, 2 추가)
def detect_risk(pose_keypoints, bed_mask, image_shape, original_shape):
    if pose_keypoints is None or len(pose_keypoints) == 0 or len(pose_keypoints[0]) < 7:
        return "Cannot assess risk. Missing or incomplete keypoints."

    if bed_mask is None:
        return "Cannot assess risk. Missing bed mask."

    # 침대 가장자리 계산
    bed_mask_resized = cv2.resize(bed_mask, (image_shape[1], image_shape[0]))
    bed_edges = np.argwhere(bed_mask_resized > 0)
    if bed_edges.size == 0:
        return "Cannot assess risk. Bed mask is empty."

    bed_min_y, bed_max_y = np.min(bed_edges[:, 0]), np.max(bed_edges[:, 0])
    bed_min_x, bed_max_x = np.min(bed_edges[:, 1]), np.max(bed_edges[:, 1])

    # 위험 단계 1: 침대 가장자리 접근
    danger_level_1 = False
    edge_threshold = 10  # 침대 가장자리로 간주할 임계값 (10픽셀)

    # 위험 단계 2: 머리/손발이 침대 경계 밖으로 나감
    danger_level_2 = False
    keypoints_to_check = [0, 9, 10, 15, 16]  # 머리(0), 손목(9, 10), 발목(15, 16)

    for idx, keypoint in enumerate(pose_keypoints[0]):
        x, y, conf = keypoint
        if conf < 0.5:  # 신뢰도 낮은 Keypoints는 생략
            continue
        x_original = x * (image_shape[1] / original_shape[1])
        y_original = y * (image_shape[0] / original_shape[0])

        # 경계 밖 여부 확인
        if not (bed_min_x <= x_original <= bed_max_x and bed_min_y <= y_original <= bed_max_y):
            if idx in keypoints_to_check:  # 머리/손발이 경계 밖으로 나감
                danger_level_2 = True
                return f"Warning Level 2: Keypoint {idx} is outside the bed!"
        else:
            # 침대 가장자리로 접근 여부 확인
            if (
                (bed_min_x + edge_threshold <= x_original <= bed_max_x - edge_threshold)
                and (bed_min_y + edge_threshold <= y_original <= bed_max_y - edge_threshold)
            ):
                continue
            else:
                danger_level_1 = True

    # 위험 단계 결과 반환
    if danger_level_2:
        return "Warning Level 2: Baby is at high risk!"
    if danger_level_1:
        return "Warning Level 1: Baby is near the edge of the bed!"

    return "No risk detected. Baby is safe."

# COCO Keypoint 연결 정의
pose_connections = [
    (0, 1), (0, 2), (1, 3), (2, 4),  # 머리 연결
    (5, 6), (5, 7), (7, 9), (6, 8), (8, 10),  # 팔 연결
    (11, 12), (11, 13), (13, 15), (12, 14), (14, 16)  # 다리 연결
]

# 뼈대 시각화 함수
def draw_pose_skeleton(image, keypoints, connections, img_shape, orig_shape):
    for connection in connections:
        start_idx, end_idx = connection
        if start_idx >= len(keypoints) or end_idx >= len(keypoints):
            continue

        # 시작점과 끝점 좌표 계산
        start_x, start_y, start_conf = keypoints[start_idx]
        end_x, end_y, end_conf = keypoints[end_idx]
        if start_conf < 0.5 or end_conf < 0.5:  # 신뢰도가 낮으면 생략
            continue

        start_x = int(start_x * (img_shape[1] / orig_shape[1]))
        start_y = int(start_y * (img_shape[0] / orig_shape[0]))
        end_x = int(end_x * (img_shape[1] / orig_shape[1]))
        end_y = int(end_y * (img_shape[0] / orig_shape[0]))

        # 유효한 Keypoint만 연결
        if start_x > 0 and start_y > 0 and end_x > 0 and end_y > 0:
            cv2.line(image, (start_x, start_y), (end_x, end_y), (0, 255, 255), 2)  # 노란색 선

    return image


# 테스트 실행
for image_path in image_paths:
    # 이미지 로드
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Unable to load image {image_path}")
        continue

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 모델 추론
    try:
        seg_results = model(image, conf=0.5, iou=0.4)  # IOU 임계값 설정
        pose_results = pose_model(image)  # Pose 추론
    except Exception as e:
        print(f"Error during inference: {e}")
        continue

    # Segmentation Masks 및 Keypoints 추출
    masks = seg_results[0].masks.data.cpu().numpy() if seg_results[0].masks is not None else []
    pose_keypoints = pose_results[0].keypoints.data.cpu().numpy() if pose_results[0].keypoints is not None else None

    # 침대 및 사람 마스크 필터링
    bed_class_id = 1  # 침대 클래스 ID
    person_class_id = 0  # 사람 클래스 ID
    bed_mask = None
    person_mask = None

    if seg_results[0].boxes is not None:
        classes = seg_results[0].boxes.cls.cpu().numpy()
        for i, cls in enumerate(classes):
            if cls == bed_class_id:
                bed_mask = masks[i]
            elif cls == person_class_id:
                person_mask = masks[i]

    # 침대 마스크 보정
    bed_mask = refine_bed_mask(image.shape, bed_mask, person_mask)

    # Segmentation 마스크 시각화
    mask_overlay = np.zeros_like(image, dtype=np.uint8)
    if bed_mask is not None:
        bed_mask_resized = cv2.resize(bed_mask, (image.shape[1], image.shape[0]))
        mask_bool = bed_mask_resized > 0.5
        mask_overlay[mask_bool] = [255, 0, 0]  # 빨간색으로 표시

        # 침대 가장자리 시각화
        bed_edges = np.argwhere(mask_bool)
        if bed_edges.size > 0:
            bed_min_y, bed_max_y = np.min(bed_edges[:, 0]), np.max(bed_edges[:, 0])
            bed_min_x, bed_max_x = np.min(bed_edges[:, 1]), np.max(bed_edges[:, 1])
            cv2.rectangle(mask_overlay, (bed_min_x, bed_min_y), (bed_max_x, bed_max_y), (255, 255, 0), 2)

    # Keypoints 및 뼈대 시각화
    blended_image = cv2.addWeighted(image, 0.7, mask_overlay, 0.3, 0)
    if pose_keypoints is not None and len(pose_keypoints[0]) > 0:
        keypoints = pose_keypoints[0]  # 첫 번째 사람의 Keypoint 사용
        # 뼈대 시각화
        blended_image = draw_pose_skeleton(blended_image, keypoints, pose_connections, image.shape, pose_results[0].orig_shape)

        # 개별 Keypoint 시각화
        for keypoint in keypoints:
            x, y, conf = keypoint
            if conf < 0.5:  # 신뢰도가 낮으면 생략
                continue
            x_original = int(x * (image.shape[1] / pose_results[0].orig_shape[1]))
            y_original = int(y * (image.shape[0] / pose_results[0].orig_shape[0]))
            cv2.circle(blended_image, (x_original, y_original), 5, (0, 255, 0), -1)  # 초록색 점

    # 위험 감지
    risk_message = detect_risk(pose_keypoints, bed_mask, image.shape, pose_results[0].orig_shape)

    # 결과 시각화
    plt.figure(figsize=(10, 10))
    plt.imshow(blended_image)
    plt.title(f"{os.path.basename(image_path)} - {risk_message}")
    plt.axis("off")
    plt.show()

    # 이미지 파일 이름 및 상태 출력
    print(f"Processed Image: {os.path.basename(image_path)}")
    print(risk_message)


Output hidden; open in https://colab.research.google.com to view.

### 뒤집기 자세 추가(윤곽선 분석 및 눈코입kp이용)

In [ ]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO

# YOLOv8 Segmentation 모델 로드
model = YOLO('/content/drive/MyDrive/Seg_dataset/best.pt')  # Segmentation 모델
pose_model = YOLO('yolov8n-pose.pt')

# 테스트 이미지 경로
test_images_dir = '/content/drive/MyDrive/baby_Prone'
image_paths = [os.path.join(test_images_dir, img) for img in os.listdir(test_images_dir) if img.endswith(('.jpg', '.png'))]

# 중심점 계산 함수
def calculate_centroid(mask):
    indices = np.argwhere(mask)  # 마스크에서 1인 픽셀 좌표를 가져옴
    if indices.size == 0:
        return None  # 비어 있는 경우 None 반환
    return np.mean(indices, axis=0)  # 좌표 평균 계산

# 침대 마스크 보정 함수
def refine_bed_mask(image_shape, bed_mask, person_mask):
    """
    침대 마스크가 없거나 작을 경우 person 마스크를 제외한 나머지를 침대 마스크로 인식.
    """
    if bed_mask is None or np.sum(bed_mask) < 0.1 * (image_shape[0] * image_shape[1]):  # 10% 이하인 경우
        print("Refining bed mask using person mask...")
        bed_mask = np.ones(image_shape[:2], dtype=np.uint8)  # 전체 영역을 1로 초기화
        if person_mask is not None:
            person_mask_resized = cv2.resize(person_mask, (image_shape[1], image_shape[0]))
            bed_mask[person_mask_resized > 0.5] = 0  # person 영역 제외
    return bed_mask

# 윤곽선 분석 기반 뒤집기 자세 감지
def detect_prone_via_contour(person_mask):
    """
    윤곽선 분석을 통해 둥근 형태(뒷통수)를 판별하여 뒤집기 자세 탐지.
    """
    if person_mask is None:
        return False, "No person mask available for contour analysis."

    contours, _ = cv2.findContours((person_mask > 0).astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) > 0:
        contour = max(contours, key=cv2.contourArea)
        convexity = cv2.isContourConvex(contour)  # 윤곽선이 둥근지 확인
        if convexity:
            return True, "Warning: Head contour detected as round (prone position)."
    return False, "Head contour not round (no prone detected)."

# 키포인트 기반 뒤집기 자세 감지
def detect_prone_via_keypoints(pose_keypoints):
    """
    얼굴 Keypoints(눈, 코)가 모두 없는 경우 뒤집기 자세로 간주.
    """
    if pose_keypoints is None or len(pose_keypoints) == 0:
        return False, "Pose keypoints missing or incomplete."

    # 얼굴 Keypoints 확인
    try:
        nose = pose_keypoints[0][0]  # 코
        left_eye = pose_keypoints[0][1]  # 왼쪽 눈
        right_eye = pose_keypoints[0][2]  # 오른쪽 눈
    except IndexError:
        return False, "Keypoints array does not contain sufficient data."

    # Keypoints의 신뢰도 확인
    if nose[2] < 0.5 and left_eye[2] < 0.5 and right_eye[2] < 0.5:
        return True, "Warning: Face keypoints missing (prone position detected)!"

    return False, "No prone position detected via keypoints."


# COCO Keypoint 연결 정의
pose_connections = [
    (0, 1), (0, 2), (1, 3), (2, 4),  # 머리 연결
    (5, 6), (5, 7), (7, 9), (6, 8), (8, 10),  # 팔 연결
    (11, 12), (11, 13), (13, 15), (12, 14), (14, 16)  # 다리 연결
]

# 뼈대 시각화 함수
def draw_pose_skeleton(image, keypoints, connections, img_shape, orig_shape):
    for connection in connections:
        start_idx, end_idx = connection
        if start_idx >= len(keypoints) or end_idx >= len(keypoints):
            continue

        # 시작점과 끝점 좌표 계산
        start_x, start_y, start_conf = keypoints[start_idx]
        end_x, end_y, end_conf = keypoints[end_idx]
        if start_conf < 0.5 or end_conf < 0.5:  # 신뢰도가 낮으면 생략
            continue

        start_x = int(start_x * (img_shape[1] / orig_shape[1]))
        start_y = int(start_y * (img_shape[0] / orig_shape[0]))
        end_x = int(end_x * (img_shape[1] / orig_shape[1]))
        end_y = int(end_y * (img_shape[0] / orig_shape[0]))

        # 유효한 Keypoint만 연결
        if start_x > 0 and start_y > 0 and end_x > 0 and end_y > 0:
            cv2.line(image, (start_x, start_y), (end_x, end_y), (0, 255, 255), 2)  # 노란색 선

    return image


# 테스트 실행
for image_path in image_paths:
    # 이미지 로드
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Unable to load image {image_path}")
        continue

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 모델 추론
    try:
        seg_results = model(image, conf=0.5, iou=0.4)  # IOU 임계값 설정
        pose_results = pose_model(image)  # Pose 추론
    except Exception as e:
        print(f"Error during inference: {e}")
        continue

    # Segmentation Masks 및 Keypoints 추출
    masks = seg_results[0].masks.data.cpu().numpy() if seg_results[0].masks is not None else []
    pose_keypoints = pose_results[0].keypoints.data.cpu().numpy() if pose_results[0].keypoints is not None else None

    # 침대 및 사람 마스크 필터링
    bed_class_id = 1  # 침대 클래스 ID
    person_class_id = 0  # 사람 클래스 ID
    bed_mask = None
    person_mask = None

    if seg_results[0].boxes is not None:
        classes = seg_results[0].boxes.cls.cpu().numpy()
        for i, cls in enumerate(classes):
            if cls == bed_class_id:
                bed_mask = masks[i]
            elif cls == person_class_id:
                person_mask = masks[i]

    # 침대 마스크 보정
    bed_mask = refine_bed_mask(image.shape, bed_mask, person_mask)

    # Segmentation 마스크 시각화
    mask_overlay = np.zeros_like(image, dtype=np.uint8)
    if bed_mask is not None:
        bed_mask_resized = cv2.resize(bed_mask, (image.shape[1], image.shape[0]))
        mask_bool = bed_mask_resized > 0.5
        mask_overlay[mask_bool] = [255, 0, 0]  # 빨간색으로 표시

        # 침대 가장자리 시각화
        bed_edges = np.argwhere(mask_bool)
        if bed_edges.size > 0:
            bed_min_y, bed_max_y = np.min(bed_edges[:, 0]), np.max(bed_edges[:, 0])
            bed_min_x, bed_max_x = np.min(bed_edges[:, 1]), np.max(bed_edges[:, 1])
            cv2.rectangle(mask_overlay, (bed_min_x, bed_min_y), (bed_max_x, bed_max_y), (255, 255, 0), 2)

    # Keypoints 및 뼈대 시각화
    blended_image = cv2.addWeighted(image, 0.7, mask_overlay, 0.3, 0)
    if pose_keypoints is not None and len(pose_keypoints[0]) > 0:
        keypoints = pose_keypoints[0]  # 첫 번째 사람의 Keypoint 사용
        # 뼈대 시각화
        blended_image = draw_pose_skeleton(blended_image, keypoints, pose_connections, image.shape, pose_results[0].orig_shape)

        # 개별 Keypoint 시각화
        for keypoint in keypoints:
            x, y, conf = keypoint
            if conf < 0.5:  # 신뢰도가 낮으면 생략
                continue
            x_original = int(x * (image.shape[1] / pose_results[0].orig_shape[1]))
            y_original = int(y * (image.shape[0] / pose_results[0].orig_shape[0]))
            cv2.circle(blended_image, (x_original, y_original), 5, (0, 255, 0), -1)  # 초록색 점

    # 위험 감지
    risk_message = detect_risk(pose_keypoints, bed_mask, image.shape, pose_results[0].orig_shape)

    # 뒤집기 자세 감지
    prone_contour_detected, contour_message = detect_prone_via_contour(person_mask)
    prone_keypoints_detected, keypoints_message = detect_prone_via_keypoints(pose_keypoints)

    # 뒤집기 자세 메시지 선택
    if prone_contour_detected:
        prone_message = contour_message
    elif prone_keypoints_detected:
        prone_message = keypoints_message
    else:
        prone_message = "No prone position detected."

    # 결과 시각화
    plt.figure(figsize=(10, 10))
    plt.imshow(blended_image)
    plt.title(f"{os.path.basename(image_path)} - {risk_message} - {prone_message}")
    plt.axis("off")
    plt.show()

    # 이미지 파일 이름 및 상태 출력
    print(f"Processed Image: {os.path.basename(image_path)}")
    print(risk_message)
    print(prone_message)


Output hidden; open in https://colab.research.google.com to view.

## 일정시간 움직임 감지 추가

In [ ]:
!pip install yt-dlp

In [ ]:
!pip install ultralytics

In [ ]:
import yt_dlp

def download_youtube_video(video_url, save_path):
    ydl_opts = {
        'format': 'bestvideo+bestaudio/best',
        'outtmpl': save_path,
        'merge_output_format': 'mp4',
        'verbose': True
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([video_url])

# 다운로드 실행
video_url_1 = "https://www.youtube.com/watch?v=yyxul1GjHmw"
download_youtube_video(video_url_1, "test_video_1.mp4")
print("Video downloaded successfully!")

video_url_2="https://www.youtube.com/watch?v=liYaasCvbwI"
download_youtube_video(video_url_2, "test_video_2.mp4")
print("Video downloaded successfully!")

video_url_5="https://www.youtube.com/watch?v=X_PgMoNZJIU"
download_youtube_video(video_url_5, "test_video_5.mp4")
print("Video downloaded successfully!")

[debug] Encodings: locale UTF-8, fs utf-8, pref UTF-8, out UTF-8 (No ANSI), error UTF-8 (No ANSI), screen UTF-8 (No ANSI)
[debug] yt-dlp version stable@2024.12.06 from yt-dlp/yt-dlp [4bd265539] (pip) API
[debug] params: {'format': 'bestvideo+bestaudio/best', 'outtmpl': 'test_video_1.mp4', 'merge_output_format': 'mp4', 'verbose': True, 'compat_opts': set(), 'http_headers': {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.15 Safari/537.36', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', 'Accept-Language': 'en-us,en;q=0.5', 'Sec-Fetch-Mode': 'navigate'}}
[debug] Python 3.10.12 (CPython x86_64 64bit) - Linux-6.1.85+-x86_64-with-glibc2.35 (OpenSSL 3.0.2 15 Mar 2022, glibc 2.35)
[debug] exe versions: ffmpeg 4.4.2 (setts), ffprobe 4.4.2
[debug] Optional libraries: certifi-2024.08.30, requests-2.32.3, secretstorage-3.3.1, sqlite3-3.37.2, urllib3-2.2.3
[debug] Proxy map: {'colab_language_server': '/us

[youtube] Extracting URL: https://www.youtube.com/watch?v=yyxul1GjHmw
[youtube] yyxul1GjHmw: Downloading webpage
[youtube] yyxul1GjHmw: Downloading ios player API JSON
[youtube] yyxul1GjHmw: Downloading mweb player API JSON


[debug] Loading youtube-nsig.f8f53e1a from cache
[debug] [youtube] Decrypted nsig LdfD4InI3ExoU59qOPL => aFzN2e-qaebQhQ
[debug] Loading youtube-nsig.f8f53e1a from cache
[debug] [youtube] Decrypted nsig 71RssNsHB_qDlspUuDk => K-yo7iuDEqKAhQ


[youtube] yyxul1GjHmw: Downloading m3u8 information


[debug] Sort order given by extractor: quality, res, fps, hdr:12, source, vcodec, channels, acodec, lang, proto
[debug] Formats sorted by: hasvid, ie_pref, quality, res, fps, hdr:12(7), source, vcodec, channels, acodec, lang, proto, size, br, asr, vext, aext, hasaud, id


...

[download] Destination: test_video_5.f248.webm
[download] 100% of    3.39MiB in 00:00:00 at 21.23MiB/s  


[debug] Invoking http downloader on "https://rr2---sn-5uaezned.googlevideo.com/videoplayback?expire=1734091724&ei=bM9bZ4OTEJy2kucPgvKqeQ&ip=35.196.31.99&id=o-AJwNkoUFgvrmNeV9C6AXE48UvvRPeiQvqjouLHJK4481&itag=251&source=youtube&requiressl=yes&xpc=EgVo2aDSNQ%3D%3D&met=1734070124%2C&mh=u7&mm=31%2C26&mn=sn-5uaezned%2Csn-a5mekn6z&ms=au%2Conr&mv=m&mvi=2&pl=22&rms=au%2Cau&initcwndbps=8560000&bui=AfMhrI_ceHFmjIs1tB5WyLqAbrY1eEcmIsQLqM0pN46EALFsOPvHQDQ8iP_qxORfQAx676nJHqAdN5LT&spc=x-caUO9myd5HimmbS9J3xw3YBK_Zk9qdaF7giFZuFBUfnmBwSw&vprv=1&svpuc=1&mime=audio%2Fwebm&rqh=1&gir=yes&clen=418628&dur=40.801&lmt=1710310547824188&mt=1734069678&fvip=4&keepalive=yes&fexp=51326932%2C51335594%2C51347747&c=IOS&txp=5318224&sparams=expire%2Cei%2Cip%2Cid%2Citag%2Csource%2Crequiressl%2Cxpc%2Cbui%2Cspc%2Cvprv%2Csvpuc%2Cmime%2Crqh%2Cgir%2Cclen%2Cdur%2Clmt&sig=AJfQdSswRQIhAL59O35CsTxBzluX6jVwubOs_IP_C7FgmAP8bBJoJ8kuAiByXpTXgII0fAxp7l6HMuSfVzCYW-F_zWL1uQFmBy3OgA%3D%3D&lsparams=met%2Cmh%2Cmm%2Cmn%2Cms%2Cmv%2Cmvi%2Cpl%

[download] Destination: test_video_5.f251.webm
[download] 100% of  408.82KiB in 00:00:00 at 8.27MiB/s   
[Merger] Merging formats into "test_video_5.mp4"


[debug] ffmpeg command line: ffmpeg -y -loglevel repeat+info -i file:test_video_5.f248.webm -i file:test_video_5.f251.webm -c copy -map 0:v:0 -map 1:a:0 -movflags +faststart file:test_video_5.temp.mp4


Deleting original file test_video_5.f251.webm (pass -k to keep)
Deleting original file test_video_5.f248.webm (pass -k to keep)
Video downloaded successfully!


In [ ]:
import cv2
import os

# 영상에서 프레임 추출 함수
def extract_frames(video_path, output_folder="frames", frame_interval=5):
    os.makedirs(output_folder, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    saved_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # 지정된 프레임 간격마다 저장
        if frame_count % frame_interval == 0:
            frame_filename = os.path.join(output_folder, f"frame_{saved_count:05d}.jpg")
            cv2.imwrite(frame_filename, frame)
            saved_count += 1

        frame_count += 1

    cap.release()
    print(f"Frames saved to {output_folder}")

# 프레임 추출 실행
extract_frames("test_video_1.mp4", output_folder="test_frames_1", frame_interval=5)
extract_frames("test_video_2.mp4", output_folder="test_frames_2", frame_interval=5)
extract_frames("test_video_5.mp4", output_folder="test_frames_5", frame_interval=5)



Frames saved to test_frames_1
Frames saved to test_frames_2
Frames saved to test_frames_5


In [ ]:
import cv2
import numpy as np

# Extract bed edges (outline only)
def extract_bed_edges(mask):
    mask = mask.astype(np.uint8) * 255  # Convert to binary image
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    edge_coords = []
    for contour in contours:
        for point in contour:
            edge_coords.append(point[0])  # Append x, y coordinates
    return edge_coords

bed_edges_list = []

for image_path in image_paths:
    # Load image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Unable to load image {image_path}")
        continue

    # Model inference
    try:
        seg_results = model(image, conf=0.5, iou=0.4)
        pose_results = pose_model(image)
    except Exception as e:
        print(f"Error during inference: {e}")
        continue

    # Extract Segmentation Masks
    masks = seg_results[0].masks.data.cpu().numpy() if seg_results[0].masks is not None else []
    pose_keypoints = pose_results[0].keypoints.data.cpu().numpy() if pose_results[0].keypoints is not None else None
    bed_mask = None

    # Filter bed mask
    if seg_results[0].boxes is not None:
        classes = seg_results[0].boxes.cls.cpu().numpy()
        for i, cls in enumerate(classes):
            if cls == 1:  # Assuming 1 is the bed class ID
                bed_mask = masks[i]

    # Process bed mask
    bed_edges = None
    if bed_mask is not None:
        bed_mask_resized = cv2.resize(bed_mask, (image.shape[1], image.shape[0]))
        mask_bool = bed_mask_resized > 0.5
        bed_edges = extract_bed_edges(mask_bool)

    # Skip if bed edges are not detected
    if bed_edges is None or len(bed_edges) == 0:
        print(f"Warning: Bed edges not detected for image {os.path.basename(image_path)}")
        continue

    # Extract keypoints and check exact match
    keypoints = []
    keypoints_on_bed_edge = 0  # Counter for keypoints exactly on the bed edge

    if pose_keypoints is not None and len(pose_keypoints[0]) > 0:
        for kp in pose_keypoints[0]:
            x, y, conf = kp
            if conf > 0.5:  # Filter based on confidence
                x_original = int(x)
                y_original = int(y)

                # Check exact match with bed edge coordinates
                exact_match = any((x_original == edge[0] and y_original == edge[1]) for edge in bed_edges)
                if exact_match:
                    keypoints_on_bed_edge += 1

                keypoints.append({"x": x_original, "y": y_original, "conf": conf, "on_bed_edge": exact_match})

    # Append data to list
    bed_edges_list.append({
        "image": os.path.basename(image_path),
        "keypoints": keypoints,
        "keypoints_on_bed_edge": keypoints_on_bed_edge
    })

# Output results
for entry in bed_edges_list:
    print(f"Image: {entry['image']}")
    if entry["keypoints"]:
        print("  Keypoints:")
        for kp in entry["keypoints"]:
            print(f"    (X: {kp['x']}, Y: {kp['y']}, Confidence: {kp['conf']}, On Bed Edge: {kp['on_bed_edge']})")
    else:
        print("  No keypoints detected.")
    print(f"  Total Keypoints On Bed Edge: {entry['keypoints_on_bed_edge']}")



0: 384x640 1 person, 9.7ms
Speed: 1.4ms preprocess, 9.7ms inference, 2.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 7.7ms
Speed: 1.4ms preprocess, 7.7ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 7.8ms
Speed: 1.4ms preprocess, 7.8ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 7.5ms
Speed: 1.3ms preprocess, 7.5ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 7.7ms
Speed: 1.3ms preprocess, 7.7ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 7.7ms
Speed: 1.3ms preprocess, 7.7ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 7.8ms
Speed: 1.1ms preprocess, 7.8ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 7.5ms
Speed: 1.4ms preprocess, 7.5ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x

#### 최종코드 + 움직임 코드

In [ ]:
import os
import shutil

# 원본 디렉토리 및 대상 디렉토리 설정
source_dir = "/content/test_frames_2"
target_dir = "/content/test_data"

# 대상 디렉토리 생성
os.makedirs(target_dir, exist_ok=True)

# frame_00100 ~ frame_00200 파일 복사
for i in range(250, 320):  # 100부터 200까지 반복
    file_name = f"frame_{i:05d}.jpg"  # 파일명 형식에 맞게 설정
    source_path = os.path.join(source_dir, file_name)
    target_path = os.path.join(target_dir, file_name)

    # 파일이 존재하면 복사
    if os.path.exists(source_path):
        shutil.copy(source_path, target_path)
        print(f"Copied: {file_name}")
    else:
        print(f"File not found: {file_name}")

print("File extraction completed!")


Copied: frame_00250.jpg
Copied: frame_00251.jpg
Copied: frame_00252.jpg
Copied: frame_00253.jpg
Copied: frame_00254.jpg
Copied: frame_00255.jpg
Copied: frame_00256.jpg
Copied: frame_00257.jpg
Copied: frame_00258.jpg
Copied: frame_00259.jpg
Copied: frame_00260.jpg
Copied: frame_00261.jpg
Copied: frame_00262.jpg
Copied: frame_00263.jpg
Copied: frame_00264.jpg
Copied: frame_00265.jpg
Copied: frame_00266.jpg
Copied: frame_00267.jpg
Copied: frame_00268.jpg
Copied: frame_00269.jpg
Copied: frame_00270.jpg
Copied: frame_00271.jpg
Copied: frame_00272.jpg
Copied: frame_00273.jpg
Copied: frame_00274.jpg
Copied: frame_00275.jpg
Copied: frame_00276.jpg
Copied: frame_00277.jpg
Copied: frame_00278.jpg
Copied: frame_00279.jpg
Copied: frame_00280.jpg
Copied: frame_00281.jpg
Copied: frame_00282.jpg
Copied: frame_00283.jpg
Copied: frame_00284.jpg
Copied: frame_00285.jpg
Copied: frame_00286.jpg
Copied: frame_00287.jpg
Copied: frame_00288.jpg
Copied: frame_00289.jpg
Copied: frame_00290.jpg
Copied: frame_00

In [ ]:
import os
import shutil

# 원본 디렉토리 및 대상 디렉토리 설정
source_dir = "/content/test_frames_5"
target_dir = "/content/test_data"

# 대상 디렉토리 생성
os.makedirs(target_dir, exist_ok=True)

# frame_00100 ~ frame_00200 파일 복사
for i in range(000, 123):  # 100부터 200까지 반복
    file_name = f"frame_{i:05d}.jpg"  # 파일명 형식에 맞게 설정
    source_path = os.path.join(source_dir, file_name)
    target_path = os.path.join(target_dir, file_name)

    # 파일이 존재하면 복사
    if os.path.exists(source_path):
        shutil.copy(source_path, target_path)
        print(f"Copied: {file_name}")
    else:
        print(f"File not found: {file_name}")

print("File extraction completed!")


Copied: frame_00000.jpg
Copied: frame_00001.jpg
Copied: frame_00002.jpg
Copied: frame_00003.jpg
Copied: frame_00004.jpg
Copied: frame_00005.jpg
Copied: frame_00006.jpg
Copied: frame_00007.jpg
Copied: frame_00008.jpg
Copied: frame_00009.jpg
Copied: frame_00010.jpg
Copied: frame_00011.jpg
Copied: frame_00012.jpg
Copied: frame_00013.jpg
Copied: frame_00014.jpg
Copied: frame_00015.jpg
Copied: frame_00016.jpg
Copied: frame_00017.jpg
Copied: frame_00018.jpg
Copied: frame_00019.jpg
Copied: frame_00020.jpg
Copied: frame_00021.jpg
Copied: frame_00022.jpg
Copied: frame_00023.jpg
Copied: frame_00024.jpg
Copied: frame_00025.jpg
Copied: frame_00026.jpg
Copied: frame_00027.jpg
Copied: frame_00028.jpg
Copied: frame_00029.jpg
Copied: frame_00030.jpg
Copied: frame_00031.jpg
Copied: frame_00032.jpg
Copied: frame_00033.jpg
Copied: frame_00034.jpg
Copied: frame_00035.jpg
Copied: frame_00036.jpg
Copied: frame_00037.jpg
Copied: frame_00038.jpg
Copied: frame_00039.jpg
Copied: frame_00040.jpg
Copied: frame_00

In [ ]:
import os
import shutil

# 원본 디렉토리 및 대상 디렉토리 설정
source_dir = "/content/test_frames_3"
target_dir = "/content/sample_3"

# 대상 디렉토리 생성
os.makedirs(target_dir, exist_ok=True)

# frame_00100 ~ frame_00200 파일 복사
for i in range(000, 139):  # 100부터 200까지 반복
    file_name = f"frame_{i:05d}.jpg"  # 파일명 형식에 맞게 설정
    source_path = os.path.join(source_dir, file_name)
    target_path = os.path.join(target_dir, file_name)

    # 파일이 존재하면 복사
    if os.path.exists(source_path):
        shutil.copy(source_path, target_path)
        print(f"Copied: {file_name}")
    else:
        print(f"File not found: {file_name}")

print("File extraction completed!")

Copied: frame_00000.jpg
Copied: frame_00001.jpg
Copied: frame_00002.jpg
Copied: frame_00003.jpg
Copied: frame_00004.jpg
Copied: frame_00005.jpg
Copied: frame_00006.jpg
Copied: frame_00007.jpg
Copied: frame_00008.jpg
Copied: frame_00009.jpg
Copied: frame_00010.jpg
Copied: frame_00011.jpg
Copied: frame_00012.jpg
Copied: frame_00013.jpg
Copied: frame_00014.jpg
Copied: frame_00015.jpg
Copied: frame_00016.jpg
Copied: frame_00017.jpg
Copied: frame_00018.jpg
Copied: frame_00019.jpg
Copied: frame_00020.jpg
Copied: frame_00021.jpg
Copied: frame_00022.jpg
Copied: frame_00023.jpg
Copied: frame_00024.jpg
Copied: frame_00025.jpg
Copied: frame_00026.jpg
Copied: frame_00027.jpg
Copied: frame_00028.jpg
Copied: frame_00029.jpg
Copied: frame_00030.jpg
Copied: frame_00031.jpg
Copied: frame_00032.jpg
Copied: frame_00033.jpg
Copied: frame_00034.jpg
Copied: frame_00035.jpg
Copied: frame_00036.jpg
Copied: frame_00037.jpg
Copied: frame_00038.jpg
Copied: frame_00039.jpg
Copied: frame_00040.jpg
Copied: frame_00

In [ ]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO

# YOLOv8 Segmentation 모델 로드
model = YOLO('/content/drive/MyDrive/Seg_dataset/best.pt')  # Segmentation 모델
pose_model = YOLO('yolov8n-pose.pt')

# 테스트 이미지 경로
test_images_dir = '/content/drive/MyDrive/baby_Prone'
image_paths = [os.path.join(test_images_dir, img) for img in os.listdir(test_images_dir) if img.endswith(('.jpg', '.png'))]
# COCO Keypoint 연결 정의
pose_connections = [
    (0, 1), (0, 2), (1, 3), (2, 4),  # 머리 연결
    (5, 6), (5, 7), (7, 9), (6, 8), (8, 10),  # 팔 연결
    (11, 12), (11, 13), (13, 15), (12, 14), (14, 16)  # 다리 연결
]

# 뼈대 시각화 함수
def draw_pose_skeleton(image, keypoints, connections, img_shape, orig_shape):
    """
    이미지에 뼈대를 시각화하는 함수
    """
    for connection in connections:
        start_idx, end_idx = connection
        if start_idx >= len(keypoints) or end_idx >= len(keypoints):
            continue

        # 시작점과 끝점 좌표 계산
        start_x, start_y, start_conf = keypoints[start_idx]
        end_x, end_y, end_conf = keypoints[end_idx]
        if start_conf < 0.5 or end_conf < 0.5:  # 신뢰도가 낮으면 생략
            continue

        start_x = int(start_x * (img_shape[1] / orig_shape[1]))
        start_y = int(start_y * (img_shape[0] / orig_shape[0]))
        end_x = int(end_x * (img_shape[1] / orig_shape[1]))
        end_y = int(end_y * (img_shape[0] / orig_shape[0]))

        # 유효한 Keypoint만 연결
        if start_x > 0 and start_y > 0 and end_x > 0 and end_y > 0:
            cv2.line(image, (start_x, start_y), (end_x, end_y), (0, 255, 255), 2)  # 노란색 선

    return image

# 키포인트 시각화 함수
def draw_keypoints(image, keypoints, img_shape, orig_shape):
    """
    이미지에 키포인트를 점으로 시각화하는 함수
    """
    for keypoint in keypoints:
        x, y, conf = keypoint
        if conf > 0.5:  # 신뢰도가 높은 키포인트만 표시
            x_original = int(x * (img_shape[1] / orig_shape[1]))
            y_original = int(y * (img_shape[0] / orig_shape[0]))
            cv2.circle(image, (x_original, y_original), 5, (0, 255, 0), -1)  # 초록색 점
    return image

# 침대 마스크 보정 함수
def refine_bed_mask(image_shape, bed_mask, person_mask):
    """
    침대 마스크가 없거나 작을 경우 person 마스크를 제외한 나머지를 침대 마스크로 인식.
    """
    if bed_mask is None or np.sum(bed_mask) < 0.1 * (image_shape[0] * image_shape[1]):  # 10% 이하인 경우
        print("Refining bed mask using person mask...")
        bed_mask = np.ones(image_shape[:2], dtype=np.uint8)  # 전체 영역을 1로 초기화
        if person_mask is not None:
            person_mask_resized = cv2.resize(person_mask, (image_shape[1], image_shape[0]))
            bed_mask[person_mask_resized > 0.5] = 0  # person 영역 제외
    return bed_mask

# 침대 테두리 좌표 추출 함수
def extract_bed_edges(mask):
    mask = mask.astype(np.uint8) * 255  # 이진화
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    edge_coords = []
    for contour in contours:
        for point in contour:
            edge_coords.append(point[0].tolist())  # (x, y) 좌표를 리스트로 추가
    return edge_coords

# 윤곽선 분석 기반 뒤집기 자세 감지
def detect_prone_via_contour(person_mask):
    """
    윤곽선 분석을 통해 둥근 형태(뒷통수)를 판별하여 뒤집기 자세 탐지.
    """
    if person_mask is None:
        return False, "No person mask available for contour analysis."

    contours, _ = cv2.findContours((person_mask > 0).astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) > 0:
        contour = max(contours, key=cv2.contourArea)
        convexity = cv2.isContourConvex(contour)  # 윤곽선이 둥근지 확인
        if convexity:
            return True, "Warning: Head contour detected as round (prone position)."
    return False, "Head contour not round (no prone detected)."

# 키포인트 기반 뒤집기 자세 감지
def detect_prone_via_keypoints(pose_keypoints):
    """
    얼굴 Keypoints(눈, 코)가 모두 없는 경우 뒤집기 자세로 간주.
    """
    if pose_keypoints is None or len(pose_keypoints) == 0:
        return False, "Pose keypoints missing or incomplete."

    try:
        nose = pose_keypoints[0][0]  # 코
        left_eye = pose_keypoints[0][1]  # 왼쪽 눈
        right_eye = pose_keypoints[0][2]  # 오른쪽 눈
    except IndexError:
        return False, "Keypoints array does not contain sufficient data."

    # Keypoints의 신뢰도 확인
    if nose[2] < 0.5 and left_eye[2] < 0.5 and right_eye[2] < 0.5:
        return True, "Warning: Face keypoints missing (prone position detected)!"

    return False, "No prone position detected via keypoints."

# 위험 감지 함수
def detect_risk_with_edges(pose_keypoints, bed_edges, bed_min_x, bed_max_x, bed_min_y, bed_max_y):
    if pose_keypoints is None or len(pose_keypoints) == 0:
        return "Cannot assess risk. Missing or incomplete keypoints."

    if not bed_edges:
        return "Cannot assess risk. Missing bed edges."

    keypoints_on_edge = 0
    keypoints_outside_bed = 0

    for keypoint in pose_keypoints[0]:
        x, y, conf = keypoint
        if conf < 0.5:  # 신뢰도 낮은 경우 생략
            continue

        x_original = int(x)
        y_original = int(y)

        # 1단계 위험: 키포인트가 침대 테두리와 정확히 동일한 위치에 있음
        if [x_original, y_original] in bed_edges:
            keypoints_on_edge += 1

        # 2단계 위험: 키포인트가 침대 외부로 나감
        if not (bed_min_x <= x_original <= bed_max_x and bed_min_y <= y_original <= bed_max_y):
            keypoints_outside_bed += 1

    if keypoints_outside_bed >= 2:
        return f"Risk Level 2: {keypoints_outside_bed} keypoints are outside the bed."
    if keypoints_on_edge > 0:
        return f"Risk Level 1: {keypoints_on_edge} keypoints are exactly on the bed edge."

    return "No risk detected."

# 테스트 실행
for image_path in image_paths:
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Unable to load image {image_path}")
        continue

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 모델 추론
    try:
        seg_results = model(image, conf=0.5, iou=0.4)  # IOU 임계값 설정
        pose_results = pose_model(image)  # Pose 추론
    except Exception as e:
        print(f"Error during inference: {e}")
        continue

    # Segmentation Masks 및 Keypoints 추출
    masks = seg_results[0].masks.data.cpu().numpy() if seg_results[0].masks is not None else []
    pose_keypoints = pose_results[0].keypoints.data.cpu().numpy() if pose_results[0].keypoints is not None else None

    # 침대 및 사람 마스크 필터링
    bed_class_id = 1
    person_class_id = 0
    bed_mask = None
    person_mask = None

    if seg_results[0].boxes is not None:
        classes = seg_results[0].boxes.cls.cpu().numpy()
        for i, cls in enumerate(classes):
            if cls == bed_class_id:
                bed_mask = masks[i]
            elif cls == person_class_id:
                person_mask = masks[i]

    # 침대 마스크 보정
    bed_mask = refine_bed_mask(image.shape, bed_mask, person_mask)

    # 침대 테두리 추출
    bed_edges = []
    bed_min_x = bed_max_x = bed_min_y = bed_max_y = None
    if bed_mask is not None:
        bed_mask_resized = cv2.resize(bed_mask, (image.shape[1], image.shape[0]))
        mask_bool = bed_mask_resized > 0.5
        bed_edges = extract_bed_edges(mask_bool)
        bed_coords = np.argwhere(mask_bool)
        if bed_coords.size > 0:
            bed_min_y, bed_max_y = np.min(bed_coords[:, 0]), np.max(bed_coords[:, 0])
            bed_min_x, bed_max_x = np.min(bed_coords[:, 1]), np.max(bed_coords[:, 1])

    # 위험 감지
    risk_message = detect_risk_with_edges(pose_keypoints, bed_edges, bed_min_x, bed_max_x, bed_min_y, bed_max_y)

    # 뒤집기 자세 감지
    prone_contour_detected, contour_message = detect_prone_via_contour(person_mask)
    prone_keypoints_detected, keypoints_message = detect_prone_via_keypoints(pose_keypoints)

    prone_message = contour_message if prone_contour_detected else keypoints_message

    # 시각화
    mask_overlay = np.zeros_like(image, dtype=np.uint8)
    if bed_mask is not None:
        mask_overlay[mask_bool] = [255, 0, 0]  # 침대 마스크 (빨간색)
        if bed_min_x is not None and bed_min_y is not None:
            cv2.rectangle(mask_overlay, (bed_min_x, bed_min_y), (bed_max_x, bed_max_y), (255, 255, 0), 2)  # 테두리 (노란색)

    blended_image = cv2.addWeighted(image, 0.7, mask_overlay, 0.3, 0)
    if pose_keypoints is not None:
        blended_image = draw_keypoints(blended_image, pose_keypoints[0], image.shape, pose_results[0].orig_shape)
        blended_image = draw_pose_skeleton(blended_image, pose_keypoints[0], pose_connections, image.shape, pose_results[0].orig_shape)

    # 텍스트 출력
    cv2.putText(blended_image, f"Risk: {risk_message}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.putText(blended_image, prone_message, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    plt.figure(figsize=(10, 10))
    plt.imshow(blended_image)
    plt.title(f"{os.path.basename(image_path)}")
    plt.axis("off")
    plt.show()

    # 로그 출력
    print(f"Processed Image: {os.path.basename(image_path)}")
    print(risk_message)
    print(prone_message)


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO

# YOLOv8 Segmentation 모델 로드
model = YOLO('/content/drive/MyDrive/Seg_dataset/best.pt')  # Segmentation 모델
pose_model = YOLO('yolov8n-pose.pt')

# 테스트 이미지 경로
test_images_dir = '/content/sample_2'
image_paths = [os.path.join(test_images_dir, img) for img in sorted(os.listdir(test_images_dir)) if img.endswith(('.jpg', '.png'))]

# COCO Keypoint 연결 정의
pose_connections = [
    (0, 1), (0, 2), (1, 3), (2, 4),  # 머리 연결
    (5, 6), (5, 7), (7, 9), (6, 8), (8, 10),  # 팔 연결
    (11, 12), (11, 13), (13, 15), (12, 14), (14, 16)  # 다리 연결
]

# 뼈대 시각화 함수
def draw_pose_skeleton(image, keypoints, connections, img_shape, orig_shape):
    for connection in connections:
        start_idx, end_idx = connection
        if start_idx >= len(keypoints) or end_idx >= len(keypoints):
            continue

        start_x, start_y, start_conf = keypoints[start_idx]
        end_x, end_y, end_conf = keypoints[end_idx]
        if start_conf < 0.5 or end_conf < 0.5:  # 신뢰도가 낮으면 생략
            continue

        start_x = int(start_x * (img_shape[1] / orig_shape[1]))
        start_y = int(start_y * (img_shape[0] / orig_shape[0]))
        end_x = int(end_x * (img_shape[1] / orig_shape[1]))
        end_y = int(end_y * (img_shape[0] / orig_shape[0]))

        cv2.line(image, (start_x, start_y), (end_x, end_y), (0, 255, 255), 2)  # 노란색 선

    return image

# 키포인트 시각화 함수
def draw_keypoints(image, keypoints, img_shape, orig_shape):
    for keypoint in keypoints:
        x, y, conf = keypoint
        if conf > 0.5:
            x_original = int(x * (img_shape[1] / orig_shape[1]))
            y_original = int(y * (img_shape[0] / orig_shape[0]))
            cv2.circle(image, (x_original, y_original), 5, (0, 255, 0), -1)  # 초록색 점
    return image

# 침대 마스크 보정 함수
def refine_bed_mask(image_shape, bed_mask, person_mask):
    if bed_mask is None or np.sum(bed_mask) < 0.3 * (image_shape[0] * image_shape[1]):
        print("Refining bed mask using person mask...")
        bed_mask = np.ones(image_shape[:2], dtype=np.uint8)
        if person_mask is not None:
            person_mask_resized = cv2.resize(person_mask, (image_shape[1], image_shape[0]))
            bed_mask[person_mask_resized > 0.5] = 0
    return bed_mask

# 침대 테두리 좌표 추출 함수
def extract_bed_edges(mask):
    mask = mask.astype(np.uint8) * 255
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    edge_coords = []
    for contour in contours:
        for point in contour:
            edge_coords.append(point[0].tolist())
    return edge_coords

# 윤곽선 분석 기반 뒤집기 자세 감지
def detect_prone_via_contour(person_mask):
    if person_mask is None:
        return False, "No person mask available for contour analysis."

    contours, _ = cv2.findContours((person_mask > 0).astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) > 0:
        contour = max(contours, key=cv2.contourArea)
        convexity = cv2.isContourConvex(contour)
        if convexity:
            return True, "Warning: Head contour detected as round (prone position)."
    return False, "Head contour not round (no prone detected)."

# 키포인트 기반 뒤집기 자세 감지
def detect_prone_via_keypoints(pose_keypoints):
    if pose_keypoints is None or len(pose_keypoints) == 0:
        return False, "Pose keypoints missing or incomplete."

    try:
        nose = pose_keypoints[0][0]
        left_eye = pose_keypoints[0][1]
        right_eye = pose_keypoints[0][2]
    except IndexError:
        return False, "Keypoints array does not contain sufficient data."

    if nose[2] < 0.5 and left_eye[2] < 0.5 and right_eye[2] < 0.5:
        return True, "Warning: prone position detected!"

    return False, "Lying position"

# 위험 감지 함수
def detect_risk_with_edges(pose_keypoints, bed_edges, bed_min_x, bed_max_x, bed_min_y, bed_max_y):
    if pose_keypoints is None or len(pose_keypoints) == 0:
        return "Cannot assess risk. Missing or incomplete keypoints."

    if not bed_edges:
        return "Cannot assess risk. Missing bed edges."

    keypoints_on_edge = 0
    keypoints_outside_bed = 0

    for keypoint in pose_keypoints[0]:
        x, y, conf = keypoint
        if conf < 0.5:
            continue

        x_original = int(x)
        y_original = int(y)

        if [x_original, y_original] in bed_edges:
            keypoints_on_edge += 1

        if not (bed_min_x <= x_original <= bed_max_x and bed_min_y <= y_original <= bed_max_y):
            keypoints_outside_bed += 1

    if keypoints_outside_bed >= 2:
        return f"Risk Level 2: {keypoints_outside_bed} keypoints are outside the bed."
    if keypoints_on_edge > 0:
        return f"Risk Level 1: {keypoints_on_edge} keypoints are exactly on the bed edge."

    return "No risk detected."

# 움직임 감지 변수 초기화
previous_keypoints = None
no_motion_time = 0
frame_interval = 1 / 30

# 테스트 실행
for image_path in image_paths:
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Unable to load image {image_path}")
        continue

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    try:
        seg_results = model(image, conf=0.5, iou=0.4)
        pose_results = pose_model(image)
    except Exception as e:
        print(f"Error during inference: {e}")
        continue

    masks = seg_results[0].masks.data.cpu().numpy() if seg_results[0].masks is not None else []
    pose_keypoints = pose_results[0].keypoints.data.cpu().numpy() if pose_results[0].keypoints is not None else None

    bed_class_id = 1
    person_class_id = 0
    bed_mask = None
    person_mask = None

    if seg_results[0].boxes is not None:
        classes = seg_results[0].boxes.cls.cpu().numpy()
        for i, cls in enumerate(classes):
            if cls == bed_class_id:
                bed_mask = masks[i]
            elif cls == person_class_id:
                person_mask = masks[i]

    bed_mask = refine_bed_mask(image.shape, bed_mask, person_mask)

    bed_edges = []
    bed_min_x = bed_max_x = bed_min_y = bed_max_y = None
    if bed_mask is not None:
        bed_mask_resized = cv2.resize(bed_mask, (image.shape[1], image.shape[0]))
        mask_bool = bed_mask_resized > 0.5
        bed_edges = extract_bed_edges(mask_bool)
        bed_coords = np.argwhere(mask_bool)
        if bed_coords.size > 0:
            bed_min_y, bed_max_y = np.min(bed_coords[:, 0]), np.max(bed_coords[:, 0])
            bed_min_x, bed_max_x = np.min(bed_coords[:, 1]), np.max(bed_coords[:, 1])

    motion_detected = False
    if pose_keypoints is not None and len(pose_keypoints) > 0 and previous_keypoints is not None and len(previous_keypoints) > 0:
        try:
            differences = np.linalg.norm(previous_keypoints[0][:, :2] - pose_keypoints[0][:, :2], axis=1)
            if np.any(differences > 5):
                motion_detected = True
                no_motion_time = 0
        except ValueError:
            print("Keypoint arrays have mismatched shapes.")
    else:
        motion_detected = True  # Assume motion if keypoints are unavailable

    if not motion_detected:
        no_motion_time += frame_interval
    else:
        no_motion_time = 0


    previous_keypoints = pose_keypoints

    risk_message = detect_risk_with_edges(pose_keypoints, bed_edges, bed_min_x, bed_max_x, bed_min_y, bed_max_y)

    prone_contour_detected, contour_message = detect_prone_via_contour(person_mask)
    prone_keypoints_detected, keypoints_message = detect_prone_via_keypoints(pose_keypoints)

    prone_detected = prone_contour_detected or prone_keypoints_detected

    if no_motion_time >= 90 and prone_detected:
        choking_status = "Choking Risk Detected!"
    else:
        choking_status = "Safe & Moving"

    mask_overlay = np.zeros_like(image, dtype=np.uint8)
    if bed_mask is not None:
        mask_overlay[mask_bool] = [255, 0, 0]
        if bed_min_x is not None and bed_min_y is not None:
            cv2.rectangle(mask_overlay, (bed_min_x, bed_min_y), (bed_max_x, bed_max_y), (255, 255, 0), 2)

    blended_image = cv2.addWeighted(image, 0.7, mask_overlay, 0.3, 0)
    if pose_keypoints is not None:
        blended_image = draw_keypoints(blended_image, pose_keypoints[0], image.shape, pose_results[0].orig_shape)
        blended_image = draw_pose_skeleton(blended_image, pose_keypoints[0], pose_connections, image.shape, pose_results[0].orig_shape)

    cv2.putText(blended_image, f"Choke: {choking_status}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        # 텍스트 출력
    cv2.putText(blended_image, f"Fall: {risk_message}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.putText(blended_image, prone_message, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    plt.figure(figsize=(10, 10))
    plt.imshow(blended_image)
    plt.title(f"{os.path.basename(image_path)}")
    plt.axis("off")
    plt.show()

    print(f"Processed Image: {os.path.basename(image_path)}")
    print(f"Choking Status: {choking_status}")


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO

# YOLOv8 Segmentation 모델 로드
model = YOLO('/content/drive/MyDrive/Seg_dataset/best.pt')  # Segmentation 모델
pose_model = YOLO('yolov8n-pose.pt')

# 테스트 이미지 경로
test_images_dir = '/content/test_frames_5'
image_paths = [os.path.join(test_images_dir, img) for img in sorted(os.listdir(test_images_dir)) if img.endswith(('.jpg', '.png'))]

# COCO Keypoint 연결 정의
pose_connections = [
    (0, 1), (0, 2), (1, 3), (2, 4),  # 머리 연결
    (5, 6), (5, 7), (7, 9), (6, 8), (8, 10),  # 팔 연결
    (11, 12), (11, 13), (13, 15), (12, 14), (14, 16)  # 다리 연결
]

# 뼈대 시각화 함수
def draw_pose_skeleton(image, keypoints, connections, img_shape, orig_shape):
    for connection in connections:
        start_idx, end_idx = connection
        if start_idx >= len(keypoints) or end_idx >= len(keypoints):
            continue

        start_x, start_y, start_conf = keypoints[start_idx]
        end_x, end_y, end_conf = keypoints[end_idx]
        if start_conf < 0.5 or end_conf < 0.5:  # 신뢰도가 낮으면 생략
            continue

        start_x = int(start_x * (img_shape[1] / orig_shape[1]))
        start_y = int(start_y * (img_shape[0] / orig_shape[0]))
        end_x = int(end_x * (img_shape[1] / orig_shape[1]))
        end_y = int(end_y * (img_shape[0] / orig_shape[0]))

        cv2.line(image, (start_x, start_y), (end_x, end_y), (0, 255, 255), 2)  # 노란색 선

    return image

# 키포인트 시각화 함수
def draw_keypoints(image, keypoints, img_shape, orig_shape):
    for keypoint in keypoints:
        x, y, conf = keypoint
        if conf > 0.5:
            x_original = int(x * (img_shape[1] / orig_shape[1]))
            y_original = int(y * (img_shape[0] / orig_shape[0]))
            cv2.circle(image, (x_original, y_original), 5, (0, 255, 0), -1)  # 초록색 점
    return image

# 침대 마스크 보정 함수
def refine_bed_mask(image_shape, bed_mask, person_mask):
    if bed_mask is None or np.sum(bed_mask) < 0.1 * (image_shape[0] * image_shape[1]):
        print("Refining bed mask using person mask...")
        bed_mask = np.ones(image_shape[:2], dtype=np.uint8)
        if person_mask is not None:
            person_mask_resized = cv2.resize(person_mask, (image_shape[1], image_shape[0]))
            bed_mask[person_mask_resized > 0.5] = 0
    return bed_mask

# 침대 테두리 좌표 추출 함수
def extract_bed_edges(mask):
    mask = mask.astype(np.uint8) * 255
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    edge_coords = []
    for contour in contours:
        for point in contour:
            edge_coords.append(point[0].tolist())
    return edge_coords

# 윤곽선 분석 기반 뒤집기 자세 감지
def detect_prone_via_contour(person_mask):
    if person_mask is None:
        return False, "No person mask available for contour analysis."

    contours, _ = cv2.findContours((person_mask > 0).astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) > 0:
        contour = max(contours, key=cv2.contourArea)
        convexity = cv2.isContourConvex(contour)
        if convexity:
            return True, "Warning: Head contour detected as round (prone position)."
    return False, "Head contour not round (no prone detected)."

# 키포인트 기반 뒤집기 자세 감지
def detect_prone_via_keypoints(pose_keypoints):
    if pose_keypoints is None or len(pose_keypoints) == 0:
        return False, "Pose keypoints missing or incomplete."

    try:
        nose = pose_keypoints[0][0]
        left_eye = pose_keypoints[0][1]
        right_eye = pose_keypoints[0][2]
    except IndexError:
        return False, "Keypoints array does not contain sufficient data."

    if nose[2] < 0.5 and left_eye[2] < 0.5 and right_eye[2] < 0.5:
        return True, "Warning: prone position detected!"

    return False, "Lying position"

# 위험 감지 함수
def detect_risk_with_edges(pose_keypoints, bed_edges, bed_min_x, bed_max_x, bed_min_y, bed_max_y):
    if pose_keypoints is None or len(pose_keypoints) == 0:
        return "Cannot assess risk. Missing or incomplete keypoints."

    if not bed_edges:
        return "Cannot assess risk. Missing bed edges."

    keypoints_on_edge = 0
    keypoints_outside_bed = 0

    for keypoint in pose_keypoints[0]:
        x, y, conf = keypoint
        if conf < 0.5:
            continue

        x_original = int(x)
        y_original = int(y)

        if [x_original, y_original] in bed_edges:
            keypoints_on_edge += 1

        if not (bed_min_x <= x_original <= bed_max_x and bed_min_y <= y_original <= bed_max_y):
            keypoints_outside_bed += 1

    if keypoints_outside_bed >= 2:
        return f"Risk Level 2: {keypoints_outside_bed} keypoints are outside the bed."
    if keypoints_on_edge > 0:
        return f"Risk Level 1: {keypoints_on_edge} keypoints are exactly on the bed edge."

    return "No risk detected."

# 움직임 감지 변수 초기화
previous_keypoints = None
no_motion_time = 0
frame_interval = 1 / 30

# 테스트 실행
for image_path in image_paths:
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Unable to load image {image_path}")
        continue

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    try:
        seg_results = model(image, conf=0.5, iou=0.4)
        pose_results = pose_model(image)
    except Exception as e:
        print(f"Error during inference: {e}")
        continue

    masks = seg_results[0].masks.data.cpu().numpy() if seg_results[0].masks is not None else []
    pose_keypoints = pose_results[0].keypoints.data.cpu().numpy() if pose_results[0].keypoints is not None else None

    bed_class_id = 1
    person_class_id = 0
    bed_mask = None
    person_mask = None

    if seg_results[0].boxes is not None:
        classes = seg_results[0].boxes.cls.cpu().numpy()
        for i, cls in enumerate(classes):
            if cls == bed_class_id:
                bed_mask = masks[i]
            elif cls == person_class_id:
                person_mask = masks[i]

    bed_mask = refine_bed_mask(image.shape, bed_mask, person_mask)

    bed_edges = []
    bed_min_x = bed_max_x = bed_min_y = bed_max_y = None
    if bed_mask is not None:
        bed_mask_resized = cv2.resize(bed_mask, (image.shape[1], image.shape[0]))
        mask_bool = bed_mask_resized > 0.5
        bed_edges = extract_bed_edges(mask_bool)
        bed_coords = np.argwhere(mask_bool)
        if bed_coords.size > 0:
            bed_min_y, bed_max_y = np.min(bed_coords[:, 0]), np.max(bed_coords[:, 0])
            bed_min_x, bed_max_x = np.min(bed_coords[:, 1]), np.max(bed_coords[:, 1])

    motion_detected = False
    if pose_keypoints is not None and len(pose_keypoints) > 0 and previous_keypoints is not None and len(previous_keypoints) > 0:
        try:
            differences = np.linalg.norm(previous_keypoints[0][:, :2] - pose_keypoints[0][:, :2], axis=1)
            if np.any(differences > 5):
                motion_detected = True
                no_motion_time = 0
        except ValueError:
            print("Keypoint arrays have mismatched shapes.")
    else:
        motion_detected = True  # Assume motion if keypoints are unavailable

    if not motion_detected:
        no_motion_time += frame_interval
    else:
        no_motion_time = 0


    previous_keypoints = pose_keypoints

    risk_message = detect_risk_with_edges(pose_keypoints, bed_edges, bed_min_x, bed_max_x, bed_min_y, bed_max_y)

    prone_contour_detected, contour_message = detect_prone_via_contour(person_mask)
    prone_keypoints_detected, keypoints_message = detect_prone_via_keypoints(pose_keypoints)

    prone_detected = prone_contour_detected or prone_keypoints_detected

    if no_motion_time >= 90 and prone_detected:
        choking_status = "Choking Risk Detected!"
    else:
        choking_status = "Safe & Moving"

    mask_overlay = np.zeros_like(image, dtype=np.uint8)
    if bed_mask is not None:
        mask_overlay[mask_bool] = [255, 0, 0]
        if bed_min_x is not None and bed_min_y is not None:
            cv2.rectangle(mask_overlay, (bed_min_x, bed_min_y), (bed_max_x, bed_max_y), (255, 255, 0), 2)

    blended_image = cv2.addWeighted(image, 0.7, mask_overlay, 0.3, 0)
    if pose_keypoints is not None:
        blended_image = draw_keypoints(blended_image, pose_keypoints[0], image.shape, pose_results[0].orig_shape)
        blended_image = draw_pose_skeleton(blended_image, pose_keypoints[0], pose_connections, image.shape, pose_results[0].orig_shape)

    cv2.putText(blended_image, f"Choke: {choking_status}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.putText(blended_image, f"Fall: {risk_message}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    if prone_contour_detected:
        cv2.putText(blended_image, contour_message, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    elif prone_keypoints_detected:
        cv2.putText(blended_image, keypoints_message, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    plt.figure(figsize=(10, 10))
    plt.imshow(blended_image)
    plt.title(f"{os.path.basename(image_path)}")
    plt.axis("off")
    plt.show()

    print(f"Processed Image: {os.path.basename(image_path)}")
    print(f"Choking Status: {choking_status}")
    print(f"Fall Status: {risk_message}")


Output hidden; open in https://colab.research.google.com to view.

#### 침대 보정 함수 제거

In [ ]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO

# YOLOv8 Segmentation 모델 로드
model = YOLO('/content/drive/MyDrive/Seg_dataset/best.pt')  # Segmentation 모델
pose_model = YOLO('yolov8n-pose.pt')

# 테스트 이미지 경로
test_images_dir = '/content/test_data'
image_paths = [os.path.join(test_images_dir, img) for img in sorted(os.listdir(test_images_dir)) if img.endswith(('.jpg', '.png'))]

# COCO Keypoint 연결 정의
pose_connections = [
    (0, 1), (0, 2), (1, 3), (2, 4),  # 머리 연결
    (5, 6), (5, 7), (7, 9), (6, 8), (8, 10),  # 팔 연결
    (11, 12), (11, 13), (13, 15), (12, 14), (14, 16)  # 다리 연결
]

# 빠대 시각화 함수
def draw_pose_skeleton(image, keypoints, connections, img_shape, orig_shape):
    for connection in connections:
        start_idx, end_idx = connection
        if start_idx >= len(keypoints) or end_idx >= len(keypoints):
            continue

        start_x, start_y, start_conf = keypoints[start_idx]
        end_x, end_y, end_conf = keypoints[end_idx]
        if start_conf < 0.5 or end_conf < 0.5:  # 신리도가 낮으면 사롭
            continue

        start_x = int(start_x * (img_shape[1] / orig_shape[1]))
        start_y = int(start_y * (img_shape[0] / orig_shape[0]))
        end_x = int(end_x * (img_shape[1] / orig_shape[1]))
        end_y = int(end_y * (img_shape[0] / orig_shape[0]))

        cv2.line(image, (start_x, start_y), (end_x, end_y), (0, 255, 255), 2)  # 노란색 선

    return image

# 키포인트 시각화 함수
def draw_keypoints(image, keypoints, img_shape, orig_shape):
    for keypoint in keypoints:
        x, y, conf = keypoint
        if conf > 0.5:
            x_original = int(x * (img_shape[1] / orig_shape[1]))
            y_original = int(y * (img_shape[0] / orig_shape[0]))
            cv2.circle(image, (x_original, y_original), 5, (0, 255, 0), -1)  # 초른색 점
    return image

# 커토를 비해 뒤집길 자세 검증
def detect_prone_via_contour(person_mask):
    if person_mask is None:
        return False, "No person mask available for contour analysis."

    contours, _ = cv2.findContours((person_mask > 0).astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) > 0:
        contour = max(contours, key=cv2.contourArea)
        convexity = cv2.isContourConvex(contour)
        if convexity:
            return True, "Warning: Head contour detected as round (prone position)."
    return False, "Head contour not round (no prone detected)."

# 키포인트 기반 뒤집길 자세 검증
def detect_prone_via_keypoints(pose_keypoints):
    if pose_keypoints is None or len(pose_keypoints) == 0:
        return False, "Pose keypoints missing or incomplete."

    try:
        nose = pose_keypoints[0][0]
        left_eye = pose_keypoints[0][1]
        right_eye = pose_keypoints[0][2]
    except IndexError:
        return False, "Keypoints array does not contain sufficient data."

    if nose[2] < 0.5 and left_eye[2] < 0.5 and right_eye[2] < 0.5:
        return True, "Warning: prone position detected!"

    return False, "Lying position"

# 위험 검증 함수
def detect_risk_with_edges(pose_keypoints, bed_edges, bed_min_x, bed_max_x, bed_min_y, bed_max_y):
    if pose_keypoints is None or len(pose_keypoints) == 0:
        return "Cannot assess risk. Missing or incomplete keypoints."

    if not bed_edges:
        return "Cannot assess risk. Missing bed edges."

    keypoints_on_edge = 0
    keypoints_outside_bed = 0

    for keypoint in pose_keypoints[0]:
        x, y, conf = keypoint
        if conf < 0.5:
            continue

        x_original = int(x)
        y_original = int(y)

        if [x_original, y_original] in bed_edges:
            keypoints_on_edge += 1

        if not (bed_min_x <= x_original <= bed_max_x and bed_min_y <= y_original <= bed_max_y):
            keypoints_outside_bed += 1

    if keypoints_outside_bed >= 2:
        return f"Risk Level 2: {keypoints_outside_bed} keypoints are outside the bed."
    if keypoints_on_edge > 0:
        return f"Risk Level 1: {keypoints_on_edge} keypoints are exactly on the bed edge."

    return "No risk detected."

# 움직임 검증 변수 초기화
previous_keypoints = None
no_motion_time = 0
frame_interval = 1 / 30

# 테스트 실행
for image_path in image_paths:
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Unable to load image {image_path}")
        continue

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    try:
        seg_results = model(image, conf=0.5, iou=0.4)
        pose_results = pose_model(image)
    except Exception as e:
        print(f"Error during inference: {e}")
        continue

    masks = seg_results[0].masks.data.cpu().numpy() if seg_results[0].masks is not None else []
    pose_keypoints = pose_results[0].keypoints.data.cpu().numpy() if pose_results[0].keypoints is not None else None

    bed_class_id = 1
    person_class_id = 0
    bed_mask = None
    person_mask = None

    if seg_results[0].boxes is not None:
        classes = seg_results[0].boxes.cls.cpu().numpy()
        for i, cls in enumerate(classes):
            if cls == bed_class_id:
                bed_mask = masks[i]
            elif cls == person_class_id:
                person_mask = masks[i]

    bed_edges = []
    bed_min_x = bed_max_x = bed_min_y = bed_max_y = None
    if bed_mask is not None:
        bed_mask_resized = cv2.resize(bed_mask, (image.shape[1], image.shape[0]))
        mask_bool = bed_mask_resized > 0.5
        bed_edges = extract_bed_edges(mask_bool)
        bed_coords = np.argwhere(mask_bool)
        if bed_coords.size > 0:
            bed_min_y, bed_max_y = np.min(bed_coords[:, 0]), np.max(bed_coords[:, 0])
            bed_min_x, bed_max_x = np.min(bed_coords[:, 1]), np.max(bed_coords[:, 1])

    motion_detected = False
    if pose_keypoints is not None and len(pose_keypoints) > 0 and previous_keypoints is not None and len(previous_keypoints) > 0:
        try:
            differences = np.linalg.norm(previous_keypoints[0][:, :2] - pose_keypoints[0][:, :2], axis=1)
            if np.any(differences > 5):
                motion_detected = True
                no_motion_time = 0
        except ValueError:
            print("Keypoint arrays have mismatched shapes.")
    else:
        motion_detected = True  # Assume motion if keypoints are unavailable

    if not motion_detected:
        no_motion_time += frame_interval
    else:
        no_motion_time = 0


    previous_keypoints = pose_keypoints

    risk_message = detect_risk_with_edges(pose_keypoints, bed_edges, bed_min_x, bed_max_x, bed_min_y, bed_max_y)

    prone_contour_detected, contour_message = detect_prone_via_contour(person_mask)
    prone_keypoints_detected, keypoints_message = detect_prone_via_keypoints(pose_keypoints)

    prone_detected = prone_contour_detected or prone_keypoints_detected

    if no_motion_time >= 90 and prone_detected:
        choking_status = "Choking Risk Detected!"
    else:
        choking_status = "Safe & Moving"

    mask_overlay = np.zeros_like(image, dtype=np.uint8)
    if bed_mask is not None:
        mask_overlay[mask_bool] = [255, 0, 0]
        if bed_min_x is not None and bed_min_y is not None:
            cv2.rectangle(mask_overlay, (bed_min_x, bed_min_y), (bed_max_x, bed_max_y), (255, 255, 0), 2)

    blended_image = cv2.addWeighted(image, 0.7, mask_overlay, 0.3, 0)
    if pose_keypoints is not None:
        blended_image = draw_keypoints(blended_image, pose_keypoints[0], image.shape, pose_results[0].orig_shape)
        blended_image = draw_pose_skeleton(blended_image, pose_keypoints[0], pose_connections, image.shape, pose_results[0].orig_shape)

    cv2.putText(blended_image, f"Choke: {choking_status}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.putText(blended_image, f"Fall: {risk_message}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    if prone_contour_detected:
        cv2.putText(blended_image, contour_message, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    elif prone_keypoints_detected:
        cv2.putText(blended_image, keypoints_message, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    plt.figure(figsize=(10, 10))
    plt.imshow(blended_image)
    plt.title(f"{os.path.basename(image_path)}")
    plt.axis("off")
    plt.show()

    print(f"Processed Image: {os.path.basename(image_path)}")
    print(f"Choking Status: {choking_status}")
    print(f"Fall Status: {risk_message}")


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import shutil

# Google Drive 경로
google_drive_path = '/content/drive/MyDrive/test_data'

# Local 경로에서 Google Drive로 이동
local_test_data_path = '/content/test_data'

# test_data 폴더를 Google Drive로 복사
shutil.copytree(local_test_data_path, google_drive_path)

print(f"Files have been moved to {google_drive_path}")



Files have been moved to /content/drive/MyDrive/test_data
